In [1]:
from rdkit import Chem
import pandas as pd
from joblib import Parallel,delayed
from rdkit.Chem import AllChem
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')

In [2]:
def runrxn(rxnsmarts,reactants):
    products=[]
    newreact=[]
    id=0
    try:
        rxn=AllChem.ReactionFromSmarts(rxnsmarts)
        AllChem.SanitizeRxn(rxn)
        rxn.Initialize()
    except:
        return False,id
    if rxn.IsInitialized():
        if len(reactants ) < rxn.GetNumReactantTemplates(): #Check if the number of reactants matches the template
            if rxn.IsMoleculeReactant(reactants[0]): #Check if molecule is a reactant in the template
                templates=rxn.GetReactants()
                for t in templates:  #Add our molecule to the reactant list in the particular position. Other reactants are the remaining reactant templates 
                    if reactants[0].HasSubstructMatch(t):
                        newreact.append(reactants[0])
                    else:
                        newreact.append(t)
                newreact=tuple(newreact)
                try:
                    ps=rxn.RunReactants(newreact)
                except:
                    return False,id
                id+=1
            else:
                ps=()
        else:
            try:
                rxn.Initialize();
                ps=rxn.RunReactants(reactants)
            except:
                return False,id
        for i in range(len(ps)):
            for j in range(len(ps[i])):
                products.append(Chem.MolToSmiles(ps[i][j]))
        return products,id
        
    else:
        return False,id

In [3]:
def check_iso(products,all_prod):
    new_prod=[]
    overall_can=[]
    red_prod=[p for p in products if (p!=[] and p!= False)]
    for x in red_prod:
        can_prod=[]
        for prod in x:
            try:
                smiles=Chem.CanonSmiles(prod)
                if smiles in overall_can: #Check for repetition across x's Eg, [[a,b],[a,c]]
                    continue
                else:
                    
                    overall_can.append(smiles)
                    if smiles in all_prod: #Checks if any element is  part of previous generations
                        can_prod.append(str(all_prod.index(smiles)))
                    else:
                        all_prod.append(smiles)
                        can_prod.append(smiles)   
            except:
                continue
        if can_prod !=[]: # Checks for repetition of x's Eg, [[a,b],[c,d],[a,b]]
            min_prod=list(set(can_prod)) #Checks for repetition in x eg.[[a,a],[b,c]]
            new_prod.append(min_prod)
        else:
            continue

    return new_prod,all_prod

In [4]:
reaction=pd.read_csv('FinalTemplate_MapRN.csv')
reactlist=reaction['Reaction Smarts'].tolist()

In [5]:
#reactants is a list of list of Smiles of the initial set of reactants
reactants=[['C(O[CH]1[CH]=[CH][CH]2[CH2]O[CH]1O2)(=O)C1[CH]=[CH][CH]=[CH][CH]=1']]
all_product=[]
f_products,all_product=check_iso(reactants,all_product)

In [ ]:
dict={}
print(f_products)
for i in range(1,4):
        products=[]
        ctr=1
        for rxns in f_products:
            ind=0
            for prod in rxns:
                ind=ind+1
                if prod.isdigit():
                    continue
                else:
                    prodlist,check = zip(*Parallel(n_jobs=-1, verbose=1)(delayed(runrxn)(r,(Chem.MolFromSmiles(prod),)) for r in reactlist))
                    #prodlist=[runrxn(r,(Chem.MolFromSmiles(prod),),i) for i,r in enumerate(reactlist)]
                    print(prod)
                    new_prod,all_product=check_iso(prodlist,all_product)
                    products.extend(new_prod)
                    string='Gen'+str(i)+str(ctr)+str(ind)
                    print(string,len(new_prod))
                    dict[string] = new_prod
                    df = pd.DataFrame({key: pd.Series(value) for key, value in dict.items()})
                    df.to_csv('Biomass/PC1_SubFPLG_Allcombo_NoRepeat_Reduced_Nghb.csv')

            ctr=ctr+1
        f_products=products

[['O=C(OC1C=CC2COC1O2)c1ccccc1']]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1048 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 9048 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 20248 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 34648 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 52248 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 73048 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 97048 tasks      | elapsed:   46.9s
[Parallel(n_jobs=-1)]: Done 124248 tasks      | elapsed:   58.9s
[Parallel(n_jobs=-1)]: Done 154648 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 188248 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 225048 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 265048 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 308248 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Do

O=C(OC1C=CC2COC1O2)c1ccccc1
Gen111 470


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 2536 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 10536 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 21736 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 36136 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 53736 tasks      | elapsed:   40.9s
[Parallel(n_jobs=-1)]: Done 74536 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 98536 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 125736 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 156136 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 189736 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 226536 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 266536 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 309736 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: D

O=C(OC1C=C(C2=CC(OC(=O)c3ccccc3)C3OCC2O3)C2COC1O2)c1ccccc1
Gen211 259


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 2536 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 10536 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 21736 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 36136 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 53736 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 74536 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 98536 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 125736 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 156136 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 189736 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 226536 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 266536 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 309736 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: D

[Li]C1(C2=CC3COC(O3)C2OC(=O)c2ccccc2)CC(OC(=O)c2ccccc2)C2OCC1O2
Gen34712 327


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1640 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 5640 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 11240 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 18440 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 27240 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 37640 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 49640 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 63240 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 78440 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 95240 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 113640 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 133640 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 155240 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 

CNC1=C(C2=CC3COC(O3)C2OC(=O)c2ccccc2)C2COC(O2)C1OC(=O)c1ccccc1
Gen34721 344


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 2536 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 10536 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 21736 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 36136 tasks      | elapsed:   31.6s


In [ ]:
for c in check:
    if c!=0:
        print('yes')